In [65]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
import keras_tuner

In [66]:
merged_data_path = '../data/v2/merged_data.jsonl'
data = pd.read_json(merged_data_path, lines=True)

In [67]:
# K-MEANS

# Prepare the list of unique genres
unique_genres = list(set([genre for genres in data['genres'] for genre in genres]) | set(
    [genre for genres in data['favourite_genres'] for genre in genres]))

# Convert the genres to a matrix of TF-IDF features
vectorizer = TfidfVectorizer()
vectorized = vectorizer.fit_transform(unique_genres)

# Apply K-means clustering
num_clusters = 100  # Adjust this value according to your needs
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
kmeans.fit(vectorized)
labels = kmeans.labels_

# Group the genres based on the cluster labels
clustered_genres = {}
for i, label in enumerate(labels):
    if label not in clustered_genres:
        clustered_genres[label] = []
    clustered_genres[label].append(unique_genres[i])

# Map genres to their cluster label
genre_to_cluster = {genre: label for label, genres in clustered_genres.items() for genre in genres}


# Define the function for mapping genres to simpler forms
def map_genre(genre):
    cluster_label = genre_to_cluster[genre]
    representative_genre = clustered_genres[cluster_label][
        0]  # Use the first genre in the cluster as the representative
    return representative_genre


# Save the genre_to_cluster and clustered_genres dictionaries to use for new data prediction
with open('../models/model2_genre_to_cluster.pickle', 'wb') as f:
    pickle.dump(genre_to_cluster, f)

with open('../models/model2_clustered_genres.pickle', 'wb') as f:
    pickle.dump(clustered_genres, f)

# Apply the mapping function to both 'genres' and 'favourite_genres' columns
data['genres'] = data['genres'].apply(lambda x: [map_genre(genre) for genre in x])
data['favourite_genres'] = data['favourite_genres'].apply(lambda x: [map_genre(genre) for genre in x])

# Delete not unique genres
data['genres'] = data['genres'].apply(lambda x: list(set(x)))
data['favourite_genres'] = data['favourite_genres'].apply(lambda x: list(set(x)))


In [68]:
# count unique genres
unique_genres = set()
for genres in data['genres']:
    unique_genres.update(genres)
for genres in data['favourite_genres']:
    unique_genres.update(genres)
print(len(unique_genres))

# how many tracks are per genre
genre_count = {}
for genres in data['genres']:
    for genre in genres:
        if genre not in genre_count:
            genre_count[genre] = 0
        genre_count[genre] += 1

genre_count = sorted(genre_count.items(), key=lambda x: x[1], reverse=True)
genre_count

100


[('lithuanian electronic', 5274),
 ('pop rock', 4674),
 ('classic hungarian pop', 3327),
 ('twee pop', 2556),
 ('hungarian folk', 1607),
 ('modern rock', 1413),
 ('belgian new wave', 956),
 ('country quebecois', 938),
 ('blues rock', 911),
 ('rap', 663),
 ('symphonic metal', 660),
 ('belly dance', 607),
 ('post-punk', 573),
 ('canadian singer-songwriter', 544),
 ('deep german hip hop', 541),
 ('philly soul', 452),
 ('uk metalcore', 375),
 ('trap argentino', 374),
 ('latin', 361),
 ('neo-progressive', 359),
 ('funk', 319),
 ('austrian punk', 284),
 ('rennes indie', 257),
 ('modern jazz piano', 242),
 ('contemporary post-bop', 204),
 ('house', 200),
 ('disco', 180),
 ('vocal house', 175),
 ('jazz', 174),
 ('synthetic classical', 163),
 ('japanese electropop', 158),
 ('brazilian edm', 145),
 ('modern power pop', 139),
 ('rock en espanol', 137),
 ('melodic hardcore', 133),
 ('rock cristiano', 130),
 ('tropical', 113),
 ('beat italiano', 109),
 ('mexican indie', 107),
 ('turkish instrumenta

In [69]:
# balance set
skipped_data = data[data["skipped"] == 1]
not_skipped_data = data[data["skipped"] == 0]

# Split the dataset into subsets based on class labels
print("skipped_data", skipped_data.shape)
print("not_skipped_data", not_skipped_data.shape)

# Calculate the minimum number of samples among all classes
min_samples = min(len(skipped_data), len(not_skipped_data))

# Randomly select samples from each class subset to match the minimum number of samples
skipped_data_balanced = skipped_data.sample(n=min_samples, random_state=42)
not_skipped_data_balanced = not_skipped_data.sample(n=min_samples, random_state=42)

# Merge the balanced subsets to create the final balanced dataset
balanced_data = pd.concat([skipped_data_balanced, not_skipped_data_balanced], axis=0)

# Shuffle the balanced dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

skipped_data (3621, 3)
not_skipped_data (6260, 3)


In [70]:
# Combine genres and favourite_genres
mlb = MultiLabelBinarizer()

data['combined_genres'] = data.apply(lambda x: x['genres'] + x['favourite_genres'], axis=1)
mlb.fit(data['combined_genres'])

encoded_favourite_genres = mlb.transform(balanced_data['favourite_genres'])
encoded_genres = mlb.transform(balanced_data['genres'])

# save mlb to file
with open('../models/model2_mlb.pickle', 'wb') as f:
    pickle.dump(mlb, f)


In [71]:
X = np.concatenate([encoded_genres, encoded_favourite_genres], axis=1)

# Extract the labels
y = balanced_data['skipped'].astype(int).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# count skipped and not skipped songs in X_train
# print("Y_train not skipped", np.count_nonzero(y_train == 0))
# print("Y_train skipped", np.count_nonzero(y_train == 1))
# print("Y_train skipped %", np.count_nonzero(y_train == 1) / len(y_train))

In [72]:
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1])))

    for i in range(hp.Int("num_layers", min_value=1, max_value=1)):
        model.add(Dense(units=hp.Int(f"units_{i}", min_value=1, max_value=3000, step=5), activation="relu"))
        model.add(Dropout(0.5))
        # if hp.Boolean("dropout"):
            # model.add(Dropout(hp.Float("dropout_rate", min_value=0.1, max_value=0.99)))

    model.add(Dense(1, activation="sigmoid"))

    # TODO hp. wybor optimizera
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [73]:
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1])))
    model.add(Dense(units=946, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation="sigmoid"))

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [74]:
epochs = 50
batch_size = 32

model = build_model(None)

model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    batch_size=batch_size,
    verbose=2
)

# tuner = keras_tuner.RandomSearch(
#     hypermodel=build_model,
#     objective="val_accuracy",
#     max_trials=50,
#     executions_per_trial=1,
#     overwrite=True,
#     directory="tuner",
#     project_name="IUM",
# )

# tuner.search_space_summary()


Epoch 1/100
182/182 - 2s - loss: 0.6767 - accuracy: 0.5816 - val_loss: 0.6512 - val_accuracy: 0.6425 - 2s/epoch - 12ms/step
Epoch 2/100
182/182 - 1s - loss: 0.6519 - accuracy: 0.6287 - val_loss: 0.6372 - val_accuracy: 0.6515 - 1s/epoch - 6ms/step
Epoch 3/100
182/182 - 1s - loss: 0.6383 - accuracy: 0.6472 - val_loss: 0.6272 - val_accuracy: 0.6549 - 1s/epoch - 6ms/step
Epoch 4/100
182/182 - 1s - loss: 0.6274 - accuracy: 0.6586 - val_loss: 0.6221 - val_accuracy: 0.6625 - 1s/epoch - 6ms/step
Epoch 5/100
182/182 - 1s - loss: 0.6231 - accuracy: 0.6579 - val_loss: 0.6200 - val_accuracy: 0.6632 - 1s/epoch - 6ms/step
Epoch 6/100
182/182 - 1s - loss: 0.6164 - accuracy: 0.6691 - val_loss: 0.6182 - val_accuracy: 0.6605 - 1s/epoch - 6ms/step
Epoch 7/100
182/182 - 1s - loss: 0.6074 - accuracy: 0.6789 - val_loss: 0.6123 - val_accuracy: 0.6605 - 1s/epoch - 6ms/step
Epoch 8/100
182/182 - 1s - loss: 0.6045 - accuracy: 0.6831 - val_loss: 0.6143 - val_accuracy: 0.6591 - 985ms/epoch - 5ms/step
Epoch 9/100


In [ ]:
# tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# tuner.results_summary()

In [ ]:
# models = tuner.get_best_models(num_models=5)
# for model in models:
#     model.summary()
# NOTATKI Z TUNINGU
# 1 layer - 946 neuronow, 0.001 learning rate (10 epok, 0.680 acc) dropout (0.5)
# 2 layer - 108,376 neuronow, 0.0015 learning rate (10 epok, 0.641 acc) dropout (0.5)
# 3 layer - 836,2576,166 neuronow, 0.0015 learning rate (10 epok, 0.652 acc) dropout (0.5)

In [ ]:
# train with best hyperparameters
# best_hp = tuner.get_best_hyperparameters()[0]
# model = build_model(best_hp)

# model.fit(
#     X_train,
#     y_train,
#     validation_data=(X_test, y_test),
#     epochs=20,
#     batch_size=32,
#     verbose=2
# )

# save model to file
model.save('../models/model2.h5')

In [ ]:
print("TEST")
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_classes))
print("Classification report:\n", classification_report(y_test, y_pred_classes))

print("TRAIN")
y_pred = model.predict(X_train)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_train, y_pred_classes))
print("Confusion matrix:\n", confusion_matrix(y_train, y_pred_classes))
print("Classification report:\n", classification_report(y_train, y_pred_classes))


TEST
46/46 [==============================] - 0s 2ms/step
Accuracy: 0.6376811594202898
Confusion matrix:
 [[484 261]
 [264 440]]
Classification report:
               precision    recall  f1-score   support

           0       0.65      0.65      0.65       745
           1       0.63      0.62      0.63       704

    accuracy                           0.64      1449
   macro avg       0.64      0.64      0.64      1449
weighted avg       0.64      0.64      0.64      1449

TRAIN
182/182 [==============================] - 0s 1ms/step
Accuracy: 0.8278957362333851
Confusion matrix:
 [[2374  502]
 [ 495 2422]]
Classification report:
               precision    recall  f1-score   support

           0       0.83      0.83      0.83      2876
           1       0.83      0.83      0.83      2917

    accuracy                           0.83      5793
   macro avg       0.83      0.83      0.83      5793
weighted avg       0.83      0.83      0.83      5793

